Load the original video of one full procedure

In [ ]:
from IPython.display import Video

Video("/Users/eddie/Trauma Thompson Demo/demos/P01_01.MP4", embed=True, width=640, height=360)

Use the first action in the video (P01_01_0 folder) (timestamp = 1 sec to 2 sec in the original video)

In [2]:
from torchvision.models.video import mvit_v2_s
import torchvision  
import torch
model = torchvision.models.video.mvit_v2_s(weights="DEFAULT")
model.head = torch.nn.Linear(768, 162)
state_dict = torch.load('mvitv2_weights.pth')
model.load_state_dict(state_dict, strict=True)

C:\Users\eddie\AppData\Local\Temp\ipykernel_85776\2753453003.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('mvitv2_weights.pth')


<All keys matched successfully>

In [ ]:
import torch
import cv2
import glob
import numpy as np
from torchvision import transforms

image_paths = sorted(glob.glob('/Users/eddie/Trauma Thompson Demo/demos/P01_01_0/*.jpg'))[:16] 

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

frames = []
for path in image_paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tensor = transform(img) 
    frames.append(tensor)

video_tensor = torch.stack(frames, dim=1)

video_tensor = video_tensor.unsqueeze(0)
print(video_tensor.shape)

torch.Size([1, 3, 16, 224, 224])


In [4]:
# Run prediction
with torch.no_grad():
    outputs = model(video_tensor)  # (1, num_classes)
    predicted_class = torch.argmax(outputs, dim=1)

print("Predicted class index:", predicted_class.item())


Predicted class index: 0
